튜토리얼 1번에서는  
1. modulo 계산만 했음. 더 실용적인 계산을 해볼 것임.  
2. Plaintext polynomial coefficient중 하나만 사용했음 (몰랐네..?) 이번엔 다 사용해볼 것.

## Modulo 결과 처리 
plain_modulus를 올리면 안 되나? --> budget consumption이 커지고, initial noise budget은 작아짐.  

In [2]:
from fase import seal
import numpy as np

## Batch encoder (for BFV only)
N: poly_modulus_degree  
T: plain_modulus 일 때,  
batching은 plaintext를  2 X (N/2) matrix로 봄.... 
음 일단 스킵 

## CKKS Encoder
BFV와 다른 점
1. plain_podulus parameter 없음.
2. coeff_modulus를 잘 설정해야함. -- tutorial4에서 추가 설명

In [3]:
parms = seal.EncryptionParameters(seal.scheme_type.ckks)

In [9]:
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, [40, 40, 40, 40, 40]))

[40, 40, 40, 40, 40] <- 이거 중요 (coeff_modulus bits)  
여기서 중요한건 prime의 정확한 값이 아니라 prime의 크기 (비트수)임 -- 나중에 ciphertext의 scale이 곱하기 연산으로 커지는 효과와 rescaling으로 prime만큼 나누어 효과로 ciphertext의 scale을 조정해줘야함  
따라서 prime 값을 입력하는게 아니라(40bit 프라임을 직접 정하기도 어렵겠지만) bit수를 입력하면 내부적으로 적당한 값을 골라줌. (seal.CoeffModulus.Create()함수)

In [10]:
context = seal.SEALContext(parms)

keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key() # 이건 create_가 아니네.
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()

In [11]:
encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

In [12]:
encoder = seal.CKKSEncoder(context)
slot_count = encoder.slot_count()

print("Number of slots:", slot_count)
# N slots = poly_modulus_degree / 2
# 하나의 slot은 하나의 숫자를 encode함. 

Number of slots: 4096


In [13]:
vector = [0.0, 1.1, 2.2, 3.3] 

**floating-point coefficient**는 'scale'만큼 확대될 것임. CKKS에서도 BFV와 마찬가지로 plaintext는 integer coefficient를 갖는 polynomial이라 생각할 수 있음. 따라서 **floating-point coefficient** (혹은 **significand**)가 달랑 한 두 자리여서는 정확한 계산이 불가능함.  

CKKS에서 메세지는 coeff_modulus의 modulo로 저장됨 (BFV에서는 plain_modulus(튜토리얼1에서 1024)의 modulo였음).  
지금 coeff_modulus는 총 200bit (40 * 5)이므로 충분한 메세지를 저장할 수 있음.  

In [19]:
scale = 2**30

ptxt = encoder.encode(vector, scale)


p_decoded=encoder.decode(ptxt)
print(p_decoded)
print(len(p_decoded))

[-4.48651637e-08  1.09999997e+00  2.19999998e+00 ... -4.97447471e-09
  1.92651125e-08 -6.42916600e-09]
4096


decode한 결과, 첫 네 자리는 vector와 (거의)같은 값이 나오고, 나머지는 0에 가까운 빈칸임.  
총 slot_count만큼의 길이.

### relinearization



In [35]:
encrypted = encryptor.encrypt(ptxt)

evaluator.square_inplace(encrypted)
evaluator.relinearize_inplace(encrypted, relin_keys)

print("Scale in the squared input: {},  {} bits".format(encrypted.scale(), np.log2(encrypted.scale())))

Scale in the squared input: 1.152921504606847e+18,  60.0 bits


In [43]:
# Decrypt할 때 scale은 알아서 보정됨. 
ptext_ = decryptor.decrypt(encrypted)
output = encoder.decode(ptext_)
print(output)
print("Error", output[:len(vector)] - np.square(vector))

[-8.59260881e-13  1.20999920e+00  4.83999952e+00 ... -2.21708419e-12
  2.48388727e-14  2.17232644e-14]
Error [-8.59260881e-13 -7.99265035e-07 -4.80676399e-07 -7.68770187e-07]


끝.